In [1]:
#Installing libraries
!pip install wikibaseintegrator

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 90 kB 5.5 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
     |████████████████████████████████| 46 kB 3.6 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [2]:
#Calling libraries
import urllib
import pandas as pd
from wikibaseintegrator import wbi_helpers
from google.colab import drive

In [ ]:
#Accessing the Google Drive
drive.mount('/content/drive')

In [ ]:
#Downloading dataset and Creating the dataframe
urllib.request.urlretrieve("https://github.com/SisonkeBiotik-Africa/AfriBioML/blob/main/CSV/keywords.csv?raw=true", "keywords.csv")
df = pd.read_csv("keywords.csv", sep=";")

In [ ]:
#Creating the Refined Dataset
with open("/content/drive/MyDrive/refinedkeywords.csv", "w") as f:
  f.write("Keyword;Class;EID;Year;Highly Cited;North Africa;OA;Journal Article\n")

Begin from here when you have already processed the dataset before.

In [ ]:
#Using the processed dataset if downloaded
df = pd.read_csv("/content/drive/MyDrive/keywords.csv", sep=";")

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 283994 entries, 0 to 291942
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Keyword          283994 non-null  object
 1   EID              283994 non-null  object
 2   Year             283994 non-null  int64 
 3   Type             283994 non-null  object
 4   Highly Cited     283994 non-null  bool  
 5   North Africa     283994 non-null  bool  
 6   OA               283994 non-null  bool  
 7   Journal Article  283994 non-null  bool  
 8   RefinedKeyword   283994 non-null  object
 9   Class            283994 non-null  object
dtypes: bool(4), int64(1), object(5)
memory usage: 16.3+ MB


In [ ]:
#Cleaning the Abstract Keywords
for item in df[df["Type"]=="Abstract"]["Keyword"].value_counts().index:
  condition = input("Is "+str(item)+" significant as a keyword [Answer: y or n]")
  if condition == "n": 
    df = df.drop(df[df["Keyword"] == item].index)
  if (condition != "n") and (condition != "y"):
    df["Keyword"] = [condition if keyword == item else keyword for keyword in df["Keyword"]]

Is follow-up significant as a keyword [Answer: y or n]y


KeyboardInterrupt: ignored

In [ ]:
df["RefinedKeyword"] = df["Keyword"]
df["Class"] = df["Keyword"]
for item in df["Keyword"].value_counts().index[0:20]:
  wikidata_id = wbi_helpers.search_entities(item, language="en", max_results=5)
  if (len(wikidata_id)>5): wikidata_id = wikidata_id[0:5]
  condition = False
  i = 0
  if (len(wikidata_id) > 0):
    while (condition == False) and (i < len(wikidata_id)):
      results = wbi_helpers.execute_sparql_query("SELECT * WHERE { {wd:"+wikidata_id[i]+" wdt:P31 ?class} UNION {wd:"+wikidata_id[i]+" wdt:P279 ?class}. wd:"+wikidata_id[i]+" rdfs:label ?label. ?class rdfs:label ?classlabel. FILTER(LANG(?label)='en') FILTER(LANG(?classlabel)='en')}")
      if (results["results"]["bindings"] != []):
        record = results["results"]["bindings"][0]
        name = record["label"]["value"]
        classes = record["classlabel"]["value"]
        c = ""
        while (c != "y") and (c != "n"):
          c = input("Does "+str(item)+" match to "+str(name)+" (Class: "+str(classes)+")? [Answer: y or n]")
          if (c == "y"):
            condition = True
            df["RefinedKeyword"] = [name if keyword == item else keyword for keyword in df["RefinedKeyword"]]
            df["Class"] = [classes if keyword == item else keyword for keyword in df["Class"]]
          else:
            i += 1
  if (condition == False):
    c = ""
    while (c == ""):
      c = input("Do you know any Wikidata ID that corresponds to "+str(item)+"? [Answer Q-id or n]")
    if (c == "n"):
      df = df.drop(df[df["Keyword"] == item].index)
    else:
      results = wbi_helpers.execute_sparql_query("SELECT * WHERE { {wd:"+c+" wdt:P31 ?class} UNION {wd:"+c+" wdt:P279 ?class}. wd:"+c+" rdfs:label ?label. ?class rdfs:label ?classlabel. FILTER(LANG(?label)='en') FILTER(LANG(?classlabel)='en')}")
      record = results["results"]["bindings"][0]
      name = record["label"]["value"]
      classes = record["classlabel"]["value"]
      df["RefinedKeyword"] = [name if keyword == item else keyword for keyword in df["RefinedKeyword"]]
      df["Class"] = [classes if keyword == item else keyword for keyword in df["Class"]]

      
      
    


Does prediction match to prediction (Class: psychic ability)? [Answer: y or n]y


Does deep learning match to deep learning (Class: machine learning method)? [Answer: y or n]y


Does accuracy match to accuracy and precision (Class: calculation)? [Answer: y or n]n


Does accuracy match to most accurate value (Class: Wikibase reason for preferred rank)? [Answer: y or n]n


Does accuracy match to most complete record (Class: Wikibase reason for preferred rank)? [Answer: y or n]n


Does accuracy match to accuracy (Class: measure)? [Answer: y or n]y


Does features match to Features (Class: album)? [Answer: y or n]n


Does features match to FEATURES (Class: scholarly article)? [Answer: y or n]n


Does features match to Features and development of Coot (Class: scholarly article)? [Answer: y or n]n


Does features match to features of Firefox (Class: Wikipedia overview article)? [Answer: y or n]n


In [ ]:
#Updating datasets
for i in df.index:
      if df["RefinedKeyword"][i] != df["Class"][i]:
        with open("/content/drive/MyDrive/refinedkeywords.csv", "a") as f:
          f.write(df["RefinedKeyword"][i]+";"+df["Class"][i]+";"+df["EID"][i]+";"+str(df["Year"][i])+";"+str(df["Highly Cited"][i])+";"+str(df["North Africa"][i])+";"+str(df["OA"][i])+";"+str(df["Journal Article"][i])+"\n")
        df = df.drop(i)

In [ ]:
#Saving the processed dataset
df.to_csv("/content/drive/MyDrive/keywords.csv", sep=";", index=False)